#### FOR A SINGLE ELEC SCAN, FINDING BIG SPIKES ####

## STEP 1: LOAD DEPENDENCIES, SET GLOBAL PARAMETERS, DEFINE FUNCTIONS

In [1]:
# Import Libraries and Library Settings
import numpy as np
import scipy.io as sio
from os.path import dirname, join as pjoin
import time
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib tk
from matplotlib.animation import FuncAnimation
import glob
import os
import re
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# Setting Params for plotting & troubleshooting
np.set_printoptions(threshold=np.inf)
plt.rcParams['animation.html']='jshtml'

# Define Functions in Use

# Split a string to determine the integer iteration, usually to find the next file (largest number)
def func(p):
    return int(((p.split('.')[0]).split('_'))[-1])

# For a histogram plotted via grid, add labels to the top of each bar
def addlabels(x,y):
    x2 = x[:-1]
    for i in range(len(x2)):
        ax4.annotate(y[i],(x2[i]+(x[1]-x[0])/2,y[i]+5),ha='center',color='r',fontsize=9)
        
# If you have a channel's row and col, return the channel's index
def map2idx(ch_row,ch_col):
    if ch_row > 31 or ch_row <0:
        print('Row out of range')
    elif ch_col >31 or ch_col<0:
        print('Col out of range')
    else:
        ch_idx = int(ch_row*32 + ch_col)
    return ch_idx

# If you have a channel index, return the channel's row and col
def idx2map(ch_idx):
    if ch_idx > 1023 or ch_idx < 0:
        print('Chan num out of range')
    else:
        ch_row = int(ch_idx/32)
        ch_col = int(ch_idx - ch_row*32)
    return (ch_row,ch_col)

## STEP 2: INITIALIZE CHANGING VARIABLES

In [2]:
piece_no = 1
time_win = 190 #The amount of time (ms) you want to view in each frame 
spike_t = 3 #How many standard deviations above noise to find spikes
real_time_win = 380 #The amount of time (ms) you want to view in the current buffer
live = False # True / False
mat_var = 'gmem1'
non_live_datarun = 0 # If looking at old data, enter in the datarun number
non_live_date_piece = '2022-01-14-1' # If looking at old data, enter in the date & piece
save = False # The resulting noise matrix and noise values, True / False

## STEP 3: DETERMINE PATHS, INITIALIZE  PLOTS, INITIALIZE VARIABLES, & FIRST CHECKS

In [5]:
# Initialize Path in Vision
path = r'/Shakedown/Users/vision/Desktop/Xilinx'

# Determine Folder and Datarun if Live Data
if live == True:
    
    date_piece = datetime.today().strftime('%Y-%m-%d') + '-' + str(piece_no)

    # Update the dataXXX folder to the latest one (assumes you want to plot noise tests happening now)
    if os.path.isdir(date_piece):
        dataruns = os.listdir(date_piece)
        last_datarun = 0
        for dir in dataruns:
            if dir.startswith('data'):
                val = int(re.findall('\d+',dir)[0])

                if val > last_datarun:
                    last_datarun = val

        run = last_datarun + 1

    else:
        run = 0

# Determine Folder and Datarun if Previously Recorded Data
elif live != True:
    date_piece = non_live_date_piece
    run = non_live_datarun
     
datarun = 'data'+str(run).zfill(3)
file_type = '/*mat'

# Initialize Figure and Properties
# Figure
plt.close('all')
fig = plt.figure(figsize=(14,12),facecolor='white',constrained_layout=False)
fig.show()
plt.style.use('fivethirtyeight')

# Grid & Subplots
grid = plt.GridSpec(2,2,width_ratios=[5,5],height_ratios=[5,5])
ax1 = plt.subplot(grid[0,0])
ax3 = plt.subplot(grid[1,0])
ax4 = plt.subplot(grid[1,1])
plt.subplots_adjust(left=0.25,right=0.94,bottom=0.05,top=0.95,wspace=0.25,hspace=0.2)

# Aesthetics
ax1.set_title('Whole Array Activity',{'fontsize':16})
ax3.set_title('Noise Matrix',{'fontsize':16})
ax4.set_title('Noise Histogram',{'fontsize':16})
textstr2 = 'Time Window = %.2f ms' %(time_win)
ax1.text(-0.8, 1.05, textstr2,transform=ax1.transAxes,fontsize=12,va = 'center')
ax1.set_xlim(left=-2,right=34)
ax1.set_ylim(bottom=-2,top=34)
textstr3 = 'GUI CONTROLS\n\n'
textstr4 = 'Pan/Zoom: Toggle [p], Click [l/r], Drag\nSave: [s]\nConstrain Pan/Zoom: Hold [x/y]\nPreserve Aspect Ratio: Hold [ctrl]\nReset: [r]'
ax1.text(-0.8,0.85,textstr3+textstr4,transform=ax1.transAxes,fontsize=9,va = 'center', ha = 'left', bbox={'facecolor':'white','edgecolor':'black','linewidth':1.5})
ax4.set_xlabel('Noise STD',{'fontsize':12})
ax4.set_ylabel('Number of Electrodes',{'fontsize':12})

# Initialize Figure Variables
max_dot = 300 #Saturation point for dot size
sample_win = int(time_win/0.05)
fig_rows_pre = 0


# Initialize Data Conversion & Analysis Variables
chan_num_pre = -1 #Setting the previous electrode in the count/time determination into an impossibility
buffer_num = 0 #Setting the first buffer incoming b/c we will be plotting all incoming buffers

noise_mean = np.zeros((32,32))
noise_std = np.zeros((32,32))
noise_cnt = np.zeros((32,32))
spike_avg = np.zeros((32,32))
spike_std = np.zeros((32,32))
spike_cnt = np.zeros((32,32))

### UNCOMMENT IF USING THE DUMMY DATA ###
#step = 1

## STEP 4: PRE-LOOP HOLDS

# If the file does not yet exist, stay here
while not(os.path.isdir(pjoin(date_piece, datarun))):
    continue

# If the file exists, but hasn't yet been filled, hold here
while True:
    data_dir = glob.glob(pjoin(date_piece,datarun)+file_type)
    if len(data_dir)>0:
        break
        

## STEP 5: CONTINUOUS DATA ANALYSIS & PLOTTING

In [6]:
while True:
    
    # Check for the Latest File
    data_dir = glob.glob(pjoin(date_piece,datarun)+file_type)
    latest_file = max(data_dir, key=func)
    
    # Initialize the File with the Next Buffer
    file_next = pjoin(date_piece,datarun,datarun+'_'+str(buffer_num)+'.mat')
    
    # If the next file to plot does not exist yet, hold here
    if func(file_next) > func(latest_file):
        continue
        
    # If the next file to plot exists but just made, hold to allow it to fill with data
    elif file_next == latest_file:
        time.sleep(1)
    
    # For the next loop, iterate
    buffer_num += 1
    
    # For current variable names
    string = mat_var
    
    ########## TESTING MULTI ELEC - UNCOMMENT IF USING DUMMY DATA ###########
#     latest_file1 = 'Dummy_Data_MultiElec_Rand.mat'
#     latest_file2 = 'Dummy_Data_MultiElec_Linear.mat'
#     latest_file3 = 'Dummy_Data_CH586_TCP_Test.mat'
    
#     if step == 1:
#         file_next = latest_file1
#         string = 'data_randnew'
#         step = 2
#     elif step == 2:
#         file_next = latest_file2
#         string = 'data_contnew'
#         step = 3
#     elif step == 3:
#         file_next = latest_file3
#         string = 'dummy_data_ch586_hard_gnd'
#         step = 1
    
    ## STEP 6: CONVERT & PROCESS DATA 
    
    # Load the next buffer file
    mat_contents = sio.loadmat(file_next)
    data = mat_contents[string][0][:]
    
    # Initialize Variables
    data_real = np.zeros((32,32,len(data)-2))  #Initialize to max possible length. Note: Throw out first two values b/c garbo
    cnt_real = np.zeros((32,32,len(data)-2))
    cnt_pre = 0 #Check for cnt changes, double cnt
    chan_index_pre = 1025 #Check for chan changes, double cnt
    N = 0 # Sample times (DOES NOT ALLOW NON-COLLISION FREE SAMPLES)
    
    for i in range(2,len(data)-1): 
        # Convert bit number into binary
        word = (np.binary_repr(data[i],32))
        
        # Break that binary into it's respective pieces and convert to bit number
        cnt = int(word[12:14],2)
        col = int(word[27:32],2)
        row = int(word[22:27],2)
        chan_index = row*32 + col
        
        # Only record the unique non-double count sample
        if(i==2 or (cnt_pre != cnt or chan_index != chan_index_pre)):
            
            # Sample time only changes when cnt changes
            if cnt != cnt_pre:
                N += 1
                
            # On the occurance the first cnt is not 0, make sure sample time is 0
            if i == 2:
                N = 0
                
            # Update variables
            cnt_pre = cnt
            chan_index_pre = chan_index
            
            # Record pertinent data
            data_real[row][col][N] = int(word[14:22],2)
            cnt_real[row][col][N] = cnt
            
    # Truncate data to be the true number of samples       
    num_samples = N
    data_real = data_real[:,:,:N]
    cnt_real = cnt_real[:,:,:N]
    
    ## STEP 7: ANALYSIS FOR PLOTTING 
    
    # AX1) Initialize Array Indices
    base = np.arange(1,33)
    rows = [ele-1 for ele in base for i in range(32)]
    temp = [1 for i in range(32)]
    cols = [ele-1 * tele for tele in temp for ele in base]

    # AX1) Initialize Variables for Array 
    size = np.zeros((32,32,0)) # For each dot, size by # of samples
    num_sam = np.zeros((32,32,1)) #Temp variable to allow sample counting
    colors = np.zeros((32,32,0)) # For each dot, color by avg amplitude
    avg_val = np.zeros((32,32,1)) #Temp variable for calculating average amplitude
    
    # AX1,AX3,AX4) Sample Counting (Note: Appends are an artifact from previous real time codes)
    num_sam[:,:,0] = np.count_nonzero(data_real,axis=2)
    incom_cnt = num_sam[:,:,0] 
    
    # AX2) Time Domain Electrodes to Plot
    fig_rows = np.count_nonzero(num_sam)
    if fig_rows == 0:
        fig_rows = 1
    if fig_rows > 4:
        fig_rows = 4
    fig_elec = np.zeros((fig_rows,2))

    fig_ind = np.argsort(num_sam.flatten())[-fig_rows:]
    fig_elec[:,0] = (fig_ind[:] / 32).astype(int)
    fig_elec[:,1] = fig_ind[:] - (fig_elec[:,0]*32)
    fig_elec = fig_elec.astype(int)
    
    # AX1,AX3,AX4) Finding average ADC of samples per electrode
    mask = np.copy(data_real)
    mask[mask==0] = np.nan
    avg_val[:,:,0] = np.nanmean(mask,axis=2)
    avg_val = np.nan_to_num(avg_val,nan=0)
    incom_mean = avg_val[:,:,0]
    
#     fig_ind = np.argsort(num_sam.flatten())[-fig_rows:]
#     fig_elec[:,0] = (fig_ind[:] / 32).astype(int)
#     fig_elec[:,1] = fig_ind[:] - (fig_elec[:,0]*32)
#     fig_elec = fig_elec.astype(int)
    
    # AX3,AX4) Finding standard deviation ADC of samples per electrode
    incom_std = np.nanstd(mask,axis=2)
    incom_std = np.nan_to_num(incom_std,nan=0)
    
    # AX3,AX4) Building standard deviation of samples per electrode as buffers come in
    pre_mean = np.copy(noise_mean)
    pre_std = np.copy(noise_std)
    pre_cnt = np.copy(noise_cnt)
    cnt_div = pre_cnt+incom_cnt
    cnt_div[cnt_div==0] = np.nan
    
    noise_mean = np.nan_to_num((pre_cnt*pre_mean + incom_cnt*incom_mean) / (cnt_div),nan=0)
    noise_std = np.sqrt(np.nan_to_num((pre_cnt*(pre_std**2+(pre_mean-noise_mean)**2)+incom_cnt*(incom_std**2+(incom_mean-noise_mean)**2))/(cnt_div),nan=0))
    noise_cnt = np.nan_to_num(pre_cnt + incom_cnt, nan=0)
    
    
    # NEW AX1)
    chan_cnt = np.count_nonzero(num_sam)
    chan_elec = np.zeros((chan_cnt,2))
    chan_ind = np.argsort(num_sam.flatten())[-chan_cnt:]
    chan_elec[:,0] = (chan_ind[:] / 32).astype(int)
    chan_elec[:,1] = chan_ind[:] - (chan_elec[:,0]*32)
    chan_elec = chan_elec.astype(int)
    
    incom_spike_cnt = np.zeros((32,32))
    incom_spike_avg = np.zeros((32,32))
    incom_spike_std = np.zeros((32,32))
    mask2 = np.copy(data_real)
    
    for x in range(len(chan_ind)):
            incom_spike_cnt[chan_elec[x,0],chan_elec[x,1]]=np.count_nonzero(mask[chan_elec[x,0],chan_elec[x,1],:]>=noise_mean[chan_elec[x,0],chan_elec[x,1]]+spike_t*noise_std[chan_elec[x,0],chan_elec[x,1]])
            mask2[chan_elec[x,0],chan_elec[x,1],mask2[chan_elec[x,0],chan_elec[x,1],:]<=noise_mean[chan_elec[x,0],chan_elec[x,1]]+spike_t*noise_std[chan_elec[x,0],chan_elec[x,1]]] = np.nan

    incom_spike_avg = np.nanmean(mask2,axis=2)
    incom_spike_avg = np.nan_to_num(incom_spike_avg,nan=0)
    incom_spike_std = np.nanstd(mask2,axis=2)
    incom_spike_std = np.nan_to_num(incom_spike_std,nan=0)
    
    pre_spike_avg = np.copy(spike_avg)
    pre_spike_std = np.copy(spike_std)
    
    pre_spike_cnt = np.copy(spike_cnt)
    new_spike_cnt = pre_spike_cnt+incom_spike_cnt
    new_spike_cnt[new_spike_cnt==0] = np.nan
    
    spike_avg = np.nan_to_num((pre_spike_cnt*pre_spike_avg + incom_spike_cnt*incom_spike_avg) / (new_spike_cnt),nan=0)
    spike_std = np.sqrt(np.nan_to_num((pre_spike_cnt*(pre_spike_std**2+(pre_spike_avg-spike_avg)**2)+incom_spike_cnt*(incom_spike_std**2+(incom_spike_avg-spike_avg)**2))/(new_spike_cnt),nan=0))
    spike_cnt = np.nan_to_num(new_spike_cnt,nan=0)
    
    # AX1) Colors by Average Electrode Amplitude
    colors = np.copy(spike_avg)
    
    # AX1) Size by Number of Samples
    scale1 = (max_dot - 15) / (np.max(spike_cnt)-np.min(spike_cnt[spike_cnt!=0]))
    b_add = max_dot - (np.max(spike_cnt)*scale1)
    size = np.round(spike_cnt*scale1+b_add)
    size[size<15] = 10

#     max_sam = np.max(num_sam[:,:,0])
#     min_sam = np.min(num_sam[num_sam!=0])
#     if max_sam == min_sam:
#         min_sam = 0
#     scale = (max_dot - 15) / (max_sam - min_sam)
#     b_add = max_dot - (max_sam*scale)
#     size = np.append(size,np.round(num_sam*scale+b_add),axis=2)
#     size[size<15] = 10
    
    # AX2) Determine the Time of Each Sample 
    total_time = num_samples*0.05 #Sampling rate 1/0.05 ms 
    times = np.linspace(0,total_time,num_samples)
    
    ## STEP 8: PLOT THAT DATA
    
    # Initialize Time Domain Subplots based on Number of Recorded Electrodes
    ax2 = grid[0,1].subgridspec(fig_rows,1)
    
    # Reset Figures if Plotting on Buffers past the first one
    if buffer_num-1 > 0:
        ax1.clear()
        cbar1.remove()
        for k in range(fig_rows_pre): #Must be to the previous buffers number of electrodes
            axs_name[k].remove()
        ax3.clear()
        ax4.clear()
        cbar.remove()
        ax1.set_title('Whole Array Activity',{'fontsize':16})
        ax3.set_title('Noise Matrix',{'fontsize':16})
        ax4.set_title('Noise Histogram',{'fontsize':16})
        textstr2 = 'Time Window = %.2f ms' %(time_win)
        ax1.text(-0.8, 1.05, textstr2,transform=ax1.transAxes,fontsize=12,va = 'center')
        ax1.set_xlim(left=-2,right=34)
        ax1.set_ylim(bottom=-2,top=34)
        textstr3 = 'GUI CONTROLS\n\n'
        textstr4 = 'Pan/Zoom: Toggle [p], Click [l/r], Drag\nSave: [s]\nConstrain Pan/Zoom: Hold [x/y]\nPreserve Aspect Ratio: Hold [ctrl]\nReset: [r]'
        ax1.text(-0.8,0.85,textstr3+textstr4,transform=ax1.transAxes,fontsize=9,va = 'center', ha = 'left', bbox={'facecolor':'white','edgecolor':'black','linewidth':1.5})
        ax4.set_xlabel('Noise STD',{'fontsize':12})
        ax4.set_ylabel('Number of Electrodes',{'fontsize':12})
    
    # Update for the next Figure Reset
    fig_rows_pre = fig_rows
    
    # Initialize Variables for Each Buffer
    x1 = rows
    y1 = cols
    x2 = np.zeros((fig_rows,1,0))
    y2 = np.zeros((fig_rows,1,0))
    tempx = np.zeros((fig_rows,1,sample_win))
    tempy = np.zeros((fig_rows,1,sample_win))
    axs_name = []*fig_rows
    
    # AX1) Plot Array
    ax1.scatter(y1,x1,s=size,c=colors,cmap='jet')
    ax1.set_aspect('equal')
    axin = inset_axes(ax1, height='90%',width='5%',loc='center right',borderpad=0)
    cbar1 = ax1.figure.colorbar(ax1.scatter(y1,x1,s=size,c=colors,cmap='jet'),ax=ax1,cax=axin,pad=0.2,orientation='vertical')
    cbar1.ax.tick_params(labelsize=9)
    cbar1.ax.set_ylabel("Amplitude (ADC)",va="bottom",fontsize=11,rotation=-90)
    
    # AX3) Plot Heatmap
    ax3.imshow(noise_std,cmap="plasma",origin='lower')
    ax3.set_aspect('equal')
    ax3.set_xticks(np.arange(data_real.shape[1]+1)-0.5,minor=True)
    ax3.set_yticks(np.arange(data_real.shape[0]+1)-0.5,minor=True)
    ax3.grid(False,which="major")
    ax3.grid(True,which="minor",axis='both',color="w",linestyle="-",linewidth=1)   
    
    # AX4) Plot Histogram
    cm = plt.cm.get_cmap('plasma')
    hist_cnt, hist_std = np.histogram(noise_std)
    hist_span = hist_std.max()-hist_std.min()
    C = [cm(((x-hist_std.min())/hist_span)) for x in hist_std]
    bar1 = ax4.bar(hist_std[:-1],hist_cnt,color=C,edgecolor="black",linewidth=2,align='edge',width = hist_std[1]-hist_std[0])
    asp = np.diff(ax4.get_xlim())[0]/np.diff(ax4.get_ylim())[0]
    ax4.set_aspect(asp)
    ax4.tick_params(axis='x',labelsize=10)
    ax4.tick_params(axis='y',labelsize=10)
    ax4.spines['left'].set_linewidth(2)
    ax4.spines['left'].set_color('black')
    ax4.spines['bottom'].set_linewidth(2)
    ax4.spines['bottom'].set_color('black')
    addlabels(hist_std,hist_cnt)
    axins = inset_axes(ax4, width="5%",height="90%",loc = 'center right',bbox_to_anchor=(0.1,0.,1,1),bbox_transform=ax4.transAxes)
    cbar = ax3.figure.colorbar(ax3.imshow(noise_std,cmap="plasma",origin='lower'),ax=ax4,cax=axins,pad=0.2)
    cbar.ax.tick_params(labelsize=9)
    cbar.ax.set_ylabel("Noise STD",rotation=-90,va="bottom",fontsize=11)
    
    # AX2) Initialize Time Domain Subplots
    for k in range(fig_rows):
        axs = fig.add_subplot(ax2[k,0])
        axs_name = np.append(axs_name,axs)
        text_elec = 'Elec # ' + str(fig_ind[k])
        axs_name[k].annotate(text_elec,(0.01,0.9),xycoords='axes fraction',va='center',fontsize = 11,color='blue')
            
    
    # AX2) Start with the last bit of information but plot all previous
    if total_time > real_time_win:
        i = num_samples - int(real_time_win/0.05)
        tempx2 = np.zeros((fig_rows,1,num_samples-int(real_time_win/0.05)-1))
        tempy2 = np.zeros((fig_rows,1,num_samples-int(real_time_win/0.05)-1))
        for k in range(fig_rows):
            tempx2[k,0,:] = times[0:num_samples-int(real_time_win/0.05)-1]
            tempy2[k,0,:] = data_real[fig_elec[k,0],fig_elec[k,1],0:num_samples-int(real_time_win/0.05)-1]
        x2 = np.append(x2,tempx2,axis=2)
        y2 = np.append(y2,tempy2,axis=2)
        for k in range(fig_rows):
            axs_name[k].plot(x2[k,0,:],y2[k,0,:],color='r',linewidth=1)
        fig.canvas.draw()
    elif total_time <= real_time_win:
        i = 0

    # AX2) Plot for the data time domain within real time window 
    while num_samples > i:
        
        # For this time window, find the next values of x & y
        for k in range(fig_rows):
            tempx[k,0,:] = times[i:i+sample_win]
            tempy[k,0,:] = data_real[fig_elec[k,0],fig_elec[k,1],i:i+sample_win]

        # Build are array variables
        x2 = np.append(x2,tempx,axis=2)
        y2 = np.append(y2,tempy,axis=2)
        
        # Plot Each Subplot
        for k in range(fig_rows):
            axs_name[k].plot(x2[k,0,:],y2[k,0,:],color='r',linewidth=1)
#             if (int(times[i])+time_win) > real_time_win:
#                 axs_name[k].set_xlim(left=max(0,int(times[len(times)]-5)-real_time_win),right=(int(times[len(times)]-1)))
#             else:
            axs_name[k].set_xlim(left=max(0,int(times[i]-5)),right=(int(times[i]))+time_win+1)
            #axs_name[k].set_ylim(bottom=0,top=250)
            axs_name[k].tick_params(axis='y',labelsize=(-2/7)*fig_rows+10+(2/7))
        
            if k == 0:
                axs_name[k].set_title('Time Domain Plots',{'fontsize':16})
                axs_name[k].tick_params(axis='x',labelsize=10)
            if k == fig_rows-1:
                axs_name[k].set_xlabel('Time (ms)', {'fontsize':12})
                axs_name[k].set_ylabel('ADC',{'fontsize':12})
            else:
                axs_name[k].set_xticklabels([])
            if fig_rows == 1:
                asp = np.diff(axs_name[0].get_xlim())[0]/np.diff(axs_name[0].get_ylim())[0]
                axs_name[0].set_aspect(asp)
                axs_name[k].spines['left'].set_linewidth(2)
                axs_name[k].spines['left'].set_color('black')
                axs_name[k].spines['bottom'].set_linewidth(2)
                axs_name[k].spines['bottom'].set_color('black')
            else:
                axs_name[k].spines['left'].set_linewidth(1)
                axs_name[k].spines['left'].set_color('black')
                axs_name[k].spines['bottom'].set_linewidth(1)
                axs_name[k].spines['bottom'].set_color('black')
                
   
            
        # Update All Plots Together and Flush
        fig.canvas.draw()
        fig.canvas.flush_events()
    
        # Iterate to the Next Time Domain Window
        i += sample_win
        
        # If at the end of the buffer data, update temp variables to do the last loop
        if i + sample_win >= len(times):
            tempx = tempx[:,:,:len(times)-i]
            tempy = tempy[:,:,:len(times)-i]
            
        # Once at the end, go to the next buffer
        if i >= num_samples:
            break
            
    if np.count_nonzero(noise_std) == 1024:
        break

if save == True:
    plt.savefig('1elec_scan_fifoF.png',facecolor=fig.get_facecolor(),transparent=True)
    noise_data = {"spike_std":spike_std,"spike_avg":spike_avg, "spike_cnt":spike_cnt}
    sio.savemat("1elec_scan_fifoF_"+date_piece+"_"+str(run)+".mat",noise_data)
            
            

<ipython-input-6-2064965928c5>:125: RuntimeWarning: Mean of empty slice
  avg_val[:,:,0] = np.nanmean(mask,axis=2)
/Users/vision/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
<ipython-input-6-2064965928c5>:125: RuntimeWarning: Mean of empty slice
  avg_val[:,:,0] = np.nanmean(mask,axis=2)
/Users/vision/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
<ipython-input-6-2064965928c5>:125: RuntimeWarning: Mean of empty slice
  avg_val[:,:,0] = np.nanmean(mask,axis=2)
/Users/vision/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
<ipython-input-6-2064965928c5>:125: RuntimeWarning: 

KeyboardInterrupt: 

In [ ]:
data_dir = glob.glob(pjoin(date_piece,datarun)+file_type)


In [ ]:
int(time[10]+time_win)

In [ ]:
times[len(times)-1]